In [1]:
using LinearAlgebra
using Jacobi
using Test

# Prime Basis

The prime basis is defined in reference element $\hat{E}=[-1,1]^2$as given in eq(2.15) of Wheeler and Yotov 2006 SIAM paper.
$$
\hat{\boldsymbol{V}}(\hat{E}) = P_1(\hat{E})^2 + r \, \text{curl} (\hat{x}^2\hat{y}) + s \, \text{curl} (\hat{x}\hat{y}^2)
$$

In [2]:
function PrimeBasis(xhat, yhat)

    """
    Input:
    xhat, yhat: are defined on the reference element Ehat=[-1,1]^2
    
    Return:
    Prime basis: of size (2,8) evaluated at xhat,yhat
    Note the first entries "2", are prime basis in the directions x,y
    "m" are the length of xhat,yhat
    "8" is the dimension of the prime basis
    """
    m = length(xhat)

    P = zeros(2,m,8)
    P[1,:,1] = ones(m)
    P[1,:,2] = xhat
    P[1,:,3] = yhat
    P[2,:,4] = ones(m)
    P[2,:,5] = xhat
    P[2,:,6] = yhat
    
    # supplement (curl term)
    P[1,:,7] = (xhat.^2)
    P[2,:,7] = (-2*xhat .* yhat)

    P[1,:,8] = (2*xhat .* yhat)
    P[2,:,8] = (-yhat.^2)
    
    return P
end

function VondermondeMat()
    """
    Input:
    ------
    Note
    3---4
    |   |
    1---2
    Output:
    ------
    VM: the 8x8 vondermonde matrix
    """
    # normals
    nl = [-1.0;0.0]
    nr = [1.0;0.0]
    nb = [0.0;-1.0]
    nt = [0.0;1.0]
    # nodes
    node1 = [-1.;-1.]
    node2 = [1.;-1.]
    node3 = [-1.;1.]
    node4 = [1.;1.]

    nodes = [node1 node2 node2 node4 node3 node4 node1 node3]
    normals = [nb nb nr nr nt nt nl nl]
    # vandermonde matrix, V_ij = phi_j(x_i).n_i
    VM = zeros(8,8)
    for i=1:8
        for j=1:8
            P = PrimeBasis([nodes[1,i]], [nodes[2,i]])
            VM[i,j] = P[1,1,j]*normals[1,i] + P[2,1,j]*normals[2,i]
        end
    end

    return VM
end

VondermondeMat (generic function with 1 method)

# Nodal Basis in libCEED
Following gives the nodal basis for quad element that we use in libCEED. Note that we have 8 dofs and since they are vector, we separate its $x,y$ componanats. Note we have not applied `Piola` map on the basis, we should apply it in Qfunction.

In [3]:
VM = VondermondeMat();
invVM = inv(VM);
V = Array{String}(undef, 2,8)
for i = 1:8
    b = zeros(8)
    b[i] = 1
    xx = invVM * b
    
    vx = Array{String}(undef, 5)
    vy = Array{String}(undef, 5)
    
    vx[1] = ""
    vx[2] = "*xhat"
    vx[3] = "*yhat"
    vx[4] = "*xhat*xhat"
    # This is 2xy, we multiplied 2 in the coefficient vector V1
    vx[5] = "*xhat*yhat"
    
    vy[1] = ""
    vy[2] = "*xhat"
    vy[3] = "*yhat"
    # This is -2xy, we multiplied -2 in the coefficient vector V2
    vy[4] = "*xhat*yhat"
    # This is -y^2, we multiplied -1 in the coefficient vector V2
    vy[5] = "*yhat*yhat"
    V1 = [xx[1] xx[2] xx[3] xx[7] 2*xx[8]]
    V2 = [xx[4] xx[5] xx[6] -2*xx[7] -xx[8]]
    
    VX = join(("$a"vx[i] for (i,a) in enumerate(V1) if a ≠ 0), " + ")
    VY = join(("$a"vy[i] for (i,a) in enumerate(V2) if a ≠ 0), " + ")
    #println("============= Nodal Basis of dof$i =============\n")
    println("Bx[$(i-1)] = ",VX, ";")
    println("By[$(i-1)] = ",VY, ";")
    #println("\n")
    
    V[1,i] = VX
    V[2,i] = VY
end

Bx[0] = -0.125 + 0.125*xhat*xhat;
By[0] = -0.25 + 0.25*xhat + 0.25*yhat + -0.25*xhat*yhat;
Bx[1] = 0.125 + -0.125*xhat*xhat;
By[1] = -0.25 + -0.25*xhat + 0.25*yhat + 0.25*xhat*yhat;
Bx[2] = 0.25 + 0.25*xhat + -0.25*yhat + -0.25*xhat*yhat;
By[2] = -0.125 + 0.125*yhat*yhat;
Bx[3] = 0.25 + 0.25*xhat + 0.25*yhat + 0.25*xhat*yhat;
By[3] = 0.125 + -0.125*yhat*yhat;
Bx[4] = -0.125 + 0.125*xhat*xhat;
By[4] = 0.25 + -0.25*xhat + 0.25*yhat + -0.25*xhat*yhat;
Bx[5] = 0.125 + -0.125*xhat*xhat;
By[5] = 0.25 + 0.25*xhat + 0.25*yhat + 0.25*xhat*yhat;
Bx[6] = -0.25 + 0.25*xhat + 0.25*yhat + -0.25*xhat*yhat;
By[6] = -0.125 + 0.125*yhat*yhat;
Bx[7] = -0.25 + 0.25*xhat + -0.25*yhat + 0.25*xhat*yhat;
By[7] = 0.125 + -0.125*yhat*yhat;


In [4]:
# this is divergence of the nodal basis that we use it in libCEED
divV = [0. 1. 0. 0. 0. 1. 0. 0.]
Dhat = divV * invVM

1×8 Matrix{Float64}:
 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25

# CeedBasisView

In [10]:
function GetBasis(xhat, yhat)
    """
    We create basis as a matrix of size (2*num_qpts,8)
    The first 1:num_qpts is dof in x-direction
    and num_qpts+1:2*num_qpts is dof in y-direction
    And
    Div operator as a matrix (num_qpts,8)
    """
    m = length(xhat)
    Bx = zeros(m,8)
    By = zeros(m,8) 
    Bx[:,1] = @. -0.125 + 0.125*xhat*xhat;
    By[:,1] = @. -0.25 + 0.25*xhat + 0.25*yhat + -0.25*xhat*yhat;
    Bx[:,2] = @. 0.125 + -0.125*xhat*xhat;
    By[:,2] = @. -0.25 + -0.25*xhat + 0.25*yhat + 0.25*xhat*yhat;
    Bx[:,3] = @. 0.25 + 0.25*xhat + -0.25*yhat + -0.25*xhat*yhat;
    By[:,3] = @. -0.125 + 0.125*yhat*yhat;
    Bx[:,4] = @. 0.25 + 0.25*xhat + 0.25*yhat + 0.25*xhat*yhat;
    By[:,4] = @. 0.125 + -0.125*yhat*yhat;
    Bx[:,5] = @. -0.125 + 0.125*xhat*xhat;
    By[:,5] = @. 0.25 + -0.25*xhat + 0.25*yhat + -0.25*xhat*yhat;
    Bx[:,6] = @. 0.125 + -0.125*xhat*xhat;
    By[:,6] = @. 0.25 + 0.25*xhat + 0.25*yhat + 0.25*xhat*yhat;
    Bx[:,7] = @. -0.25 + 0.25*xhat + 0.25*yhat + -0.25*xhat*yhat;
    By[:,7] = @. -0.125 + 0.125*yhat*yhat;
    Bx[:,8] = @. -0.25 + 0.25*xhat + -0.25*yhat + 0.25*xhat*yhat;
    By[:,8] = @. 0.125 + -0.125*yhat*yhat

    B = zeros(2m,8)
    B[1:m,:] = Bx[1:m,:]
    B[m+1:2*m,:] = By[1:m,:]
    
    Dhat = zeros(1,8)
    Dhat[1,:] .= 0.25;
    Div = repeat(Dhat, inner=(m,1))
    
    return B, Div
end

GetBasis (generic function with 1 method)

In [11]:
function GetQuadrature2D(Q, quad_mode)
    """ 
    Input:
    Q: number of quadrature points in 1D over [-1,1]
    quad_mode: GAUSS or LOBATTO
    Return:Gauss Quadrature data over [-1,1]^2.
    w: weights of quadrature pts
    qx: quadrature pts in x
    qy: quadrature pts in y 
    """
    
    # 1D Gauss
    if quad_mode == "GAUSS"
        q = zgj(Q, 0.0, 0.0)
        w1 = wgj(q, 0.0, 0.0)
    elseif quad_mode == "LOBATTO"
        q = zglj(Q, 0.0, 0.0)
        w1 = wglj(q, 0.0, 0.0)
    end
    w = zeros(Q*Q)
    qx = zeros(Q*Q)
    qy = zeros(Q*Q)
    for i=1:Q
        for j=1:Q
            k = (i-1)*Q +j
            qx[k] = q[j]
            qy[k] = q[i]
            w[k] = w1[j]*w1[i]
        end
    end
    return w, qx, qy
end

GetQuadrature2D (generic function with 1 method)

In [15]:
Q = 3
num_qpts = Q*Q
w2, qx, qy = GetQuadrature2D(Q, "GAUSS")
B, Div = GetBasis(qx, qy);

In [16]:
B

18×8 Matrix{Float64}:
 -0.05        0.05        0.1        …  0.05       -0.787298   -0.1
 -0.125       0.125       0.443649      0.125      -0.443649   -0.0563508
 -0.05        0.05        0.787298      0.05       -0.1        -0.0127017
 -0.05        0.05        0.0563508     0.05       -0.443649   -0.443649
 -0.125       0.125       0.25          0.125      -0.25       -0.25
 -0.05        0.05        0.443649   …  0.05       -0.0563508  -0.0563508
 -0.05        0.05        0.0127017     0.05       -0.1        -0.787298
 -0.125       0.125       0.0563508     0.125      -0.0563508  -0.443649
 -0.05        0.05        0.1           0.05       -0.0127017  -0.1
 -0.787298   -0.1        -0.05          0.0127017  -0.05        0.05
 -0.443649   -0.443649   -0.05       …  0.0563508  -0.05        0.05
 -0.1        -0.787298   -0.05          0.1        -0.05        0.05
 -0.443649   -0.0563508  -0.125         0.0563508  -0.125       0.125
 -0.25       -0.25       -0.125         0.25       -0.1

In [17]:
Div

9×8 Matrix{Float64}:
 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25
 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25
 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25
 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25
 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25
 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25
 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25
 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25
 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25

In [21]:
# Just to check the values with libCEED, copied the result of t330-basis.c for Q=3, and CEED_GAUSS
B_CEED = [
-0.05000000  0.05000000  0.10000000 0.01270167 -0.05000000 0.05000000 -0.78729833 -0.10000000;
-0.12500000  0.12500000  0.44364917 0.05635083 -0.12500000 0.12500000 -0.44364917 -0.05635083;
-0.05000000  0.05000000  0.78729833 0.10000000 -0.05000000 0.05000000 -0.10000000 -0.01270167;
-0.05000000  0.05000000  0.05635083 0.05635083 -0.05000000 0.05000000 -0.44364917 -0.44364917;
-0.12500000  0.12500000  0.25000000 0.25000000 -0.12500000 0.12500000 -0.25000000 -0.25000000;
-0.05000000  0.05000000  0.44364917 0.44364917 -0.05000000 0.05000000 -0.05635083 -0.05635083;
-0.05000000  0.05000000  0.01270167 0.10000000 -0.05000000 0.05000000 -0.10000000 -0.78729833;
-0.12500000  0.12500000  0.05635083 0.44364917 -0.12500000 0.12500000 -0.05635083 -0.44364917;
-0.05000000  0.05000000  0.10000000 0.78729833 -0.05000000 0.05000000 -0.01270167 -0.10000000;
-0.78729833 -0.10000000 -0.05000000 0.05000000  0.10000000 0.01270167 -0.05000000  0.05000000;
-0.44364917 -0.44364917 -0.05000000 0.05000000  0.05635083 0.05635083 -0.05000000  0.05000000;
-0.10000000 -0.78729833 -0.05000000 0.05000000  0.01270167 0.10000000 -0.05000000  0.05000000;
-0.44364917 -0.05635083 -0.12500000 0.12500000  0.44364917 0.05635083 -0.12500000  0.12500000;
-0.25000000 -0.25000000 -0.12500000 0.12500000  0.25000000 0.25000000 -0.12500000  0.12500000;
-0.05635083 -0.44364917 -0.12500000 0.12500000  0.05635083 0.44364917 -0.12500000  0.12500000;
-0.10000000 -0.01270167 -0.05000000 0.05000000  0.78729833 0.10000000 -0.05000000  0.05000000;
-0.05635083 -0.05635083 -0.05000000 0.05000000  0.44364917 0.44364917 -0.05000000  0.05000000;
-0.01270167 -0.10000000 -0.05000000 0.05000000  0.10000000 0.78729833 -0.05000000  0.05000000
];

In [22]:
@test B_CEED ≈ B

Test Passed